This file forecasts derailment/subreddit using BERT.

# 1. Install requirements

In [31]:
model_name = "bert-base-uncased"
test_size = 0.2
labels = [True, False]  # replace with your real labels
num_labels = len(labels)
id2label = {i: label for i, label in enumerate(labels)}
label2id = {label: i for i, label in enumerate(labels)}

# 2. Read and Prepare the Dataset

In [32]:
import pandas as pd
import pickle
import os

In [33]:
corpus = os.path.join('..', '..', 'Data', 'processed', 'email_label_dict_balanced.pkl')

In [34]:
# Load the balanced dictionary
with open(corpus, 'rb') as f:
    emails_and_tag = pickle.load(f)

print(f"Loaded {len(emails_and_tag):,} emails")

# Check class distribution
casual_count = sum(1 for v in emails_and_tag.values() if v == 0)
business_count = sum(1 for v in emails_and_tag.values() if v == 1)
print(f"Casual: {casual_count:,} | Business: {business_count:,}")
print(f"Ratio: 1:{business_count/casual_count:.1f}")


Loaded 20,640 emails
Casual: 10,320 | Business: 10,320
Ratio: 1:1.0


In [35]:
import random

emails = []
is_business = []

for key, value in emails_and_tag.items():
    emails.append(key)
    is_business.append(value) 


# Zip the lists together
combined = list(zip(emails, is_business))

# Shuffle the combined list
random.shuffle(combined)

# Unzip back into two lists
email_shuffled, is_buisness_shuffled = zip(*combined)

# Convert back to lists (optional)
email_shuffled = list(email_shuffled)
is_buisness_shuffled = list(is_buisness_shuffled)

pre_dataframe = {"email": email_shuffled, "labels":is_buisness_shuffled}
df = pd.DataFrame(pre_dataframe)

In [36]:
df.head()

,email,labels
0,Subject: GE FORM-FYI\n\n----------------------...,0
1,Subject: invoices\n\nnever received a pass word.,1
2,Subject: RE: Drinks\n\ni'm going to be in the ...,0
3,"Subject: Re: Lead Time\n\nIn that case, defini...",1
4,Subject: Preview the Newest Winter Essentials\...,1


### Label Encoder

In [37]:
from sklearn import preprocessing

In [38]:
# le = preprocessing.LabelEncoder()
# le.fit(df["derails"].tolist())
# df['label'] = le.transform(df["derails"].tolist())

In [39]:
df.head()

,email,labels
0,Subject: GE FORM-FYI\n\n----------------------...,0
1,Subject: invoices\n\nnever received a pass word.,1
2,Subject: RE: Drinks\n\ni'm going to be in the ...,0
3,"Subject: Re: Lead Time\n\nIn that case, defini...",1
4,Subject: Preview the Newest Winter Essentials\...,1


### Train/Test/Val Split

In [40]:
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict

In [41]:
# First split off 20% for test
df_train_val, df_test = train_test_split(df, test_size=0.2, random_state=42)

# Now split the remaining 80% into 60% train and 20% val
# 20% out of the remaining 80% is 0.25 of that chunk
df_train, df_val = train_test_split(df_train_val, test_size=0.25, random_state=42)

# Hugging Face format
dataset = DatasetDict({
    "train": Dataset.from_pandas(df_train.reset_index(drop=True)),
    "validation": Dataset.from_pandas(df_val.reset_index(drop=True)),
    "test": Dataset.from_pandas(df_test.reset_index(drop=True)),
})


### Convert to correct dataframe

In [42]:
train_dataset = dataset["train"]
val_dataset = dataset["validation"]
test_dataset = dataset["test"]


### Tokenizer

In [43]:
from transformers import AutoTokenizer

In [44]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

def preprocess_function(examples):
    return tokenizer(examples["email"], truncation=True)

In [45]:
tokenized_train = train_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 12384/12384 [00:03<00:00, 3201.97 examples/s]


In [46]:
tokenized_test = test_dataset.map(preprocess_function, batched=True)

Map: 100%|██████████| 4128/4128 [00:00<00:00, 4781.76 examples/s]


# 3. Initialize Model

In [47]:
from transformers import AutoModelForSequenceClassification

In [48]:
def model_init():
    return AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels)

# 4. Train model

In [49]:
from transformers import DataCollatorWithPadding
import evaluate
import numpy as np

In [50]:
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [51]:
metric = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    acc = metric.compute(predictions=predictions, references=labels)["accuracy"]
    return {"eval_accuracy": acc}

In [52]:
from transformers import TrainingArguments, Trainer, EarlyStoppingCallback

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=10,
    logging_strategy="epoch",
    eval_strategy="epoch",                 # ✅ clean & modern
    save_strategy="epoch",                 # ✅ matches eval
    save_total_limit=2,                    # ✅ keeps best 2 checkpoints
    load_best_model_at_end=True,           # ✅ will restore best version
    metric_for_best_model="eval_accuracy", # ✅ or use "eval_accuracy"
    greater_is_better=True,                # ✅ for accuracy
    report_to="none",
    learning_rate=3e-5,
    per_device_train_batch_size=128,       # for speed
    per_device_eval_batch_size=256,        # speed
    fp16=True,                             # speed
    warmup_steps=500,
    weight_decay=0.05,
)

trainer = Trainer(
    # model=model,
    model_init=model_init,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]  # 👈 Add this line
)


/tmp/ipykernel_2306607/2788371201.py:22: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [53]:
def compute_objective(metrics):
    return metrics["eval_accuracy"]


Now re-generate the model with the best hyperparameters

In [54]:
# 4. Retrain and save
trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.624100,0.477536,0.801599
2,0.411900,0.369642,0.854409
3,0.324100,0.306453,0.876696
4,0.251800,0.259932,0.894138
5,0.190500,0.274352,0.891231
6,0.128500,0.316591,0.887597
7,0.081100,0.349711,0.888566
8,0.045200,0.396753,0.897771
9,0.026000,0.427223,0.902616
10,0.013700,0.450054,0.899467


TrainOutput(global_step=970, training_loss=0.2097072495627649, metrics={'train_runtime': 323.8358, 'train_samples_per_second': 382.416, 'train_steps_per_second': 2.995, 'total_flos': 3.25836730957824e+16, 'train_loss': 0.2097072495627649, 'epoch': 10.0})

In [55]:
# trainer.save_model('BERT')

# 5. Evaluate Model

In [56]:
from sklearn.metrics import classification_report

In [57]:
preds = trainer.predict(tokenized_train)
preds = np.argmax(preds[:3][0],axis=1)
GT = df_train['labels'].tolist()
print(classification_report(GT,preds))

              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6191
           1       1.00      1.00      1.00      6193

    accuracy                           1.00     12384
   macro avg       1.00      1.00      1.00     12384
weighted avg       1.00      1.00      1.00     12384



In [58]:
output = trainer.predict(tokenized_test)
logits = output.predictions  # raw model output before argmax
preds = np.argmax(logits, axis=1)  # class with highest score

GT = df_val['labels'].tolist()  
print(classification_report(GT, preds))


              precision    recall  f1-score   support

           0       0.50      0.48      0.49      2050
           1       0.50      0.52      0.51      2078

    accuracy                           0.50      4128
   macro avg       0.50      0.50      0.50      4128
weighted avg       0.50      0.50      0.50      4128



In [59]:
trainer.evaluate()

{'eval_accuracy': 0.9026162790697675,
 'eval_loss': 0.42722323536872864,
 'eval_runtime': 3.9092,
 'eval_samples_per_second': 1055.961,
 'eval_steps_per_second': 4.349,
 'epoch': 10.0}

In [60]:
trainer.save_model("./my_email_classifier")
tokenizer.save_pretrained("./my_email_classifier")

('./my_email_classifier/tokenizer_config.json',
 './my_email_classifier/special_tokens_map.json',
 './my_email_classifier/vocab.txt',
 './my_email_classifier/added_tokens.json',
 './my_email_classifier/tokenizer.json')